Get all Wikidata objects and checks if they are connected to an OSM object using an [API](https://osm.wikidata.link/tagged)

* this [Notebook](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/OSM_Wikidata.ipynb)

* API [Wikidata to OpenStreetMap](https://osm.wikidata.link/tagged)
  * eg. [https://osm.wikidata.link/tagged/api/item/Q106708773](https://osm.wikidata.link/tagged/api/item/Q106708773)

* Another tool [osm.wikidata.link](https://osm.wikidata.link/search)
  

TODO: 
* 


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-08-05 00:31:18.644475


In [2]:
import pandas as pd


In [3]:
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
#
queryWD = """SELECT DISTINCT (REPLACE(STR(?site), ".*Q", "Q") AS ?qid) ?site WHERE {
  #?site wdt:P17 wd:Q33. 
  #?site wdt:P17/wdt:P30 wd:Q46. #Europe
  ?site wdt:P31/wdt:P279* wd:Q3914.
  ?site wdt:P625 ?coordinates.
   minus {
    { ?site wdt:P10689 ?OSMid. }
    UNION
    { ?site wdt:P402 ?OSMrelid. }
    UNION 
    { ?site wdt:P11693 ?OSMnode. }
  }
} limit 200000"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDo = get_sparql_dataframe(endpoint_url, queryWD )
WDo["Source"] = "WD"     
WDo.shape

(200000, 3)

In [4]:
WDo.head()

,qid,site,Source
0,Q318880,http://www.wikidata.org/entity/Q318880,WD
1,Q12654912,http://www.wikidata.org/entity/Q12654912,WD
2,Q12659978,http://www.wikidata.org/entity/Q12659978,WD
3,Q12661362,http://www.wikidata.org/entity/Q12661362,WD
4,Q12677508,http://www.wikidata.org/entity/Q12677508,WD


In [5]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listWDo = []
for WD, row in tqdm(WDo.iterrows(), total=WDo.shape[0]):
    url = "https://osm.wikidata.link/tagged/api/item/" + row["qid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["qid"] 
    try:
        r = http.request('GET', url) 
        data = json.loads(r.data.decode('utf-8'))
    except:
        print (r.status, url)
    #print (r.status)
    try:
        #print(data)
        osmid = data["osm"][0]["id"]            
        osmtype = data["osm"][0]["type"]              
    except:
        #print ("error")
        #print(data)
        osmid =""
        osmtype =""
    new_item['osmid'] = osmid  
    new_item['type'] = osmtype 
    
    listWDo.append(new_item)
print (len(listWDo))

 22%|██▏       | 43184/200000 [2:33:00<556:55:58, 12.79s/it]

502 https://osm.wikidata.link/tagged/api/item/Q6811795


 22%|██▏       | 43185/200000 [2:34:00<1028:43:08, 23.62s/it]

502 https://osm.wikidata.link/tagged/api/item/Q6811796


 22%|██▏       | 43186/200000 [2:35:00<1421:14:13, 32.63s/it]

...

502 https://osm.wikidata.link/tagged/api/item/Q7054886


 22%|██▏       | 43283/200000 [4:12:16<2624:58:58, 60.30s/it]

502 https://osm.wikidata.link/tagged/api/item/Q7054895


100%|██████████| 200000/200000 [11:54:39<00:00,  4.66it/s]   

200000


In [6]:
OSMtot = pd.DataFrame(listWDo,
                  columns=['wikidata','type','osmid'])
OSMtot.shape


(200000, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,type,osmid
0,Q318880,,
1,Q12654912,,
2,Q12659978,,
3,Q12661362,,
4,Q12677508,,
5,Q12680306,,
6,Q16470479,,
7,Q16476066,,
8,Q16476363,,
9,Q16476441,,


In [8]:
#OSMempty = OSMtot.osmid.notnull()
OSMtot[(OSMtot['osmid']=="")].shape

(163634, 3)

In [9]:
OSMEmpty =OSMtot[(OSMtot['osmid']=="")]

In [10]:
OSMEmpty.shape

(163634, 3)

In [11]:
OSMEmpty.to_csv("WD - OSM school 3 missing.csv")

OSMEmpty.head()

,wikidata,type,osmid
0,Q318880,,
1,Q12654912,,
2,Q12659978,,
3,Q12661362,,
4,Q12677508,,


In [12]:
OSMConnected=OSMtot[(OSMtot['osmid']!="")]
OSMConnected.to_csv("WD - OSM_school 3.csv")
OSMConnected.head()

,wikidata,type,osmid
231,Q7380723,node,354242485
284,Q2895377,node,1586863947
286,Q2898551,node,2572336843
487,Q5904530,way,130535965
489,Q5913254,way,10045363


In [13]:
print("*", start_time.strftime("%Y%m%d"),"WD objects", WDo.shape[0], "ej OSM kopplade",OSMEmpty.shape[0]) 


* 20230805 WD objects 200000 ej OSM kopplade 163634


Generate Markdown table eg.
| 20210526     | 2802 | 2050 |1147 | 254 | 213| 84|


In [14]:
print("|",start_time.strftime("%Y%m%d"),"|", \
      WDo.shape[0],"|",OSMEmpty.wikidata.nunique(),"|")


| 20230805 | 200000 | 163634 |


In [15]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-08-05 12:26:22.830326
Time elapsed (hh:mm:ss.ms) 11:55:04.186381
